# **TP Intelligence Artificielle - SAT et CSP**

# **Partie 0 : Visualisation**

Voici une fonction pour visualiser les sudokus.

In [1]:
from IPython.display import HTML

def visualize_sudoku(grid):
    html = "<table>"
    for i in range(9):
        html += "<tr>"
        for j in range(9):
            value = grid[i][j]
            cell_style = "width: 30px; height: 30px; text-align: center; font-size: 20px; border: 1px solid black;"
            if i % 3 == 0 and i != 0:  # Horizontal lines
                cell_style += "border-top: 5px solid black;"
            if j % 3 == 0 and j != 0:  # Vertical lines
                cell_style += "border-left: 5px solid black;"

            if value == 0:
                html += f"<td style='background-color: white; {cell_style}'> </td>"  # Case vide
            else:
                html += f"<td style='background-color: rgb(226, 114, 91); {cell_style}'>{value}</td>"
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

### **Exercice 1**
Écrivez un Sudoku sous forme d’une liste de listes (9 listes de 9 élements chacune où la valeur 0 represente une caise vide), puis utilisez la fonction visualize_sudoku pour l’afficher.

In [2]:

grid = [
    [0, 0, 0, 7, 0, 0, 0, 0, 0],
    [0, 0, 7, 0, 5, 1, 0, 0, 0],
    [9, 0, 5, 0, 3, 8, 0, 0, 7],
    [0, 9, 0, 5, 0, 7, 0, 1, 0],
    [4, 0, 1, 0, 0, 0, 3, 6, 0],
    [2, 0, 0, 1, 0, 0, 0, 7, 0],
    [0, 0, 4, 0, 8, 0, 0, 0, 3],
    [0, 3, 0, 0, 7, 0, 0, 0, 2],
    [0, 0, 6, 0, 0, 0, 0, 0, 0]
]
visualize_sudoku(grid)

,,,7,,,,,
,,7,,5,1,,,
9,,5,,3,8,,,7
,9,,5,,7,,1,
4,,1,,,,3,6,
2,,,1,,,,7,
,,4,,8,,,,3
,3,,,7,,,,2
,,6,,,,,,


# **Partie 1 : Modélisation**

Nous allons modéliser le Sudoku comme un problème SAT et CSP, puis utiliser les méthodes propres à chaque modélisation pour la résolution.
Pour cela, nous utiliserons deux bibliothèques : **python-sat** et **python-constraint**.

Exécutez le bloc de code suivant pour les installer.
Pour les installer sur vos PC, il suffit d’exécuter les mêmes lignes (sans le '!' au début) dans une console de commandes (CMD).



In [3]:
# Installation des libraries
# pip install python-sat
# pip install python-constraint

## **1.1 Modélisation SAT**

Pour modéliser le Sudoku comme un problème SAT, nous utiliserons la méthode CNF de la bibliothèque pysat.formula.
CNF permet de définir les clauses logiques représentant les contraintes du problème.
Pour les variables, nous utiliserons un dictionnaire.

In [ ]:
from pysat.formula import CNF

class Sudoku_SAT:
    # 将 9x9 Sudoku 编码为 SAT（CNF）问题
    # var_map[(i, j, k)] = 变量编号，表示“第 i 行第 j 列取值 k”
    def __init__(self, grid):
        self.grid = grid
        self.cnf = CNF()  # CNF 子句容器
        self.var_map = {}  # (行, 列, 值) -> SAT 变量 ID

        # 1) 创建 SAT 变量映射
        var_count = 1
        for i in range(9):
            for j in range(9):
                for k in range(1, 10):
                    self.var_map[(i, j, k)] = var_count
                    var_count += 1

        # 2) 生成 CNF 约束子句

        # a) 每个格子至少有一个值
        for i in range(9):
            for j in range(9):
                clause = [self.var_map[(i, j, k)] for k in range(1, 10)]
                self.cnf.append(clause)

        # b) 每个格子至多有一个值
        for i in range(9):
            for j in range(9):
                for k1 in range(1, 10):
                    for k2 in range(k1 + 1, 10):
                        clause = [-self.var_map[(i, j, k1)], -self.var_map[(i, j, k2)]]
                        #(- X_{i,j,k1} 或 - X_{i,j,k2})
                        # 表示“第 i 行第 j 列不能同时取 k1 和 k2”
                        self.cnf.append(clause)

        # c) 每一行必须包含 1..9（至少一次） 
        for i in range(9):
            for k in range(1, 10):
                clause = [self.var_map[(i, j, k)] for j in range(9)]
                self.cnf.append(clause)

        # d) 每一列必须包含 1..9（至少一次）
        for j in range(9):
            for k in range(1, 10):
                clause = [self.var_map[(i, j, k)] for i in range(9)]
                self.cnf.append(clause)

        # e) 每个 3x3 宫必须包含 1..9（至少一次）
        for subgrid_row in range(3):
            for subgrid_col in range(3):
                for k in range(1, 10):
                    clause = [
                        self.var_map[(subgrid_row * 3 + i, subgrid_col * 3 + j, k)]
                        # 表示“第 -- 宫 第 i 行第 j 列取值 k”
                        for i in range(3)
                        for j in range(3)
                    ]
                    self.cnf.append(clause)

        # f) 初始已知数字约束（单位子句）
        for i in range(9):
            for j in range(9):
                if self.grid[i][j] != 0:
                    self.cnf.append([self.var_map[(i, j, self.grid[i][j])]])


### **Exercice 2**
Créez une fonction qui prend une grille de Sudoku en entrée et utilise le solveur Glucose3 de la bibliothèque pysat.solver pour résoudre le jeu (pensez à bien importer Glucosa3). Votre fonction doit :

1. Créer un objet sudoku_sat pour modéliser le problème.
2. Ajouter au solveur les contraintes du Sudoku sous forme de clauses logiques.
3. Vérifier si le Sudoku est résoluble en utilisant le solveur SAT.
4. Trouver une solution si le Sudoku est résoluble.
5. Interpréter la solution retournée par le solveur pour remplir la grille :
  * La solution sera une liste de taille 9 × 9 × 10, contenant des valeurs positives et négatives.
  * Une valeur positive signifie que la variable (i, j, k) est True, ce qui indique que la case (i, j) doit contenir la valeur k.

In [ ]:
from pysat.solvers import Glucose3

def solve_sudoku_sat(grid):
    """
    使用 SAT 求解器 (Glucose3) 解决数独问题。
    """
    # 1. 创建 sudoku_sat 对象来建模问题 (生成 CNF 子句)
    model = Sudoku_SAT(grid)
    
    # 2. 初始化求解器
    # 使用 'with' 语句可以确保求解器在使用后正确关闭释放资源
    with Glucose3() as solver:
        # 3. 将数独的约束 (CNF 子句) 添加到求解器中
        solver.append_formula(model.cnf.clauses)
        
        # 4. 检查数独是否可解
        if solver.solve():
            # 如果可解，获取模型 (即让所有子句为真的变量赋值列表)
            solution = solver.get_model() #获取答案
            solution_set = set(solution)  # 转换为集合以加快查找速度
            
            # 5. 解释解决方案并填充网格
            # 遍历我们在 Sudoku_SAT 中创建的映射: (行, 列, 值) -> var_id
            for (i, j, k), var_id in model.var_map.items():
                # 意味着在第 i 行第 j 列填入数字 k
                if var_id in solution_set: #如果 var_id 在解决方案中 (即该变量为 True)
                    grid[i][j] = k
            
            return grid
        else:
            print("Pas de solution trouvée (无解)")
            return None

### **Exercice 3**
Utilisez votre fonction pour resoudre le sudoku ci-dessous:

    [0, 0, 0, 0, 7, 0, 0, 0, 0]
    [6, 0, 0, 1, 9, 5, 0, 0, 0]
    [0, 9, 8, 0, 0, 0, 0, 6, 0]
    [8, 0, 0, 0, 6, 0, 0, 0, 3]
    [4, 0, 0, 8, 0, 3, 0, 0, 1]
    [7, 0, 0, 0, 2, 0, 0, 0, 6]
    [0, 6, 0, 0, 0, 0, 2, 8, 0]
    [0, 0, 0, 4, 1, 9, 0, 0, 5]
    [0, 0, 0, 0, 8, 0, 0, 7, 9]

In [6]:
# SAT 调用示例：先定义待求解网格，再调用 solve_sudoku_sat
grid = [
    [0, 0, 0, 7, 0, 0, 0, 0, 0],
    [0, 0, 7, 0, 5, 1, 0, 0, 0],
    [9, 0, 5, 0, 3, 8, 0, 0, 7],
    [0, 9, 0, 5, 0, 7, 0, 1, 0],
    [4, 0, 1, 0, 0, 0, 3, 6, 0],
    [2, 0, 0, 1, 0, 0, 0, 7, 0],
    [0, 0, 4, 0, 8, 0, 0, 0, 3],
    [0, 3, 0, 0, 7, 0, 0, 0, 2],
    [0, 0, 6, 0, 0, 0, 0, 0, 0],
]

res = solve_sudoku_sat(grid)
if res:
    visualize_sudoku(res)
else:
    print("aucune solution")


8,6,2,7,9,4,5,3,1
3,4,7,2,5,1,8,9,6
9,1,5,6,3,8,4,2,7
6,9,3,5,4,7,2,1,8
4,7,1,8,2,9,3,6,5
2,5,8,1,6,3,9,7,4
7,2,4,9,8,6,1,5,3
1,3,9,4,7,5,6,8,2
5,8,6,3,1,2,7,4,9


## **1.2 Modélisation CSP**

### **Exercice 4**

Créez une fonction solve_sudoku_csp qui prend une grille de Sudoku en entrée, la modélise comme un problème CSP et la résout. Votre fonction doit :
1. Créer une instance du problème CSP
  * Utilisez la classe Problem de la bibliothèque python-constraint pour représenter le Sudoku.
2. Définir les variables du problème
  * Utilisez la méthode addVariable pour ajouter les variables au problème, sachant que :
    * Chaque case de la grille est une variable identifiée par un tuple (ligne, colonne).
    * Le domaine de chaque variable correspond aux valeurs possibles (de 1 à 9).
    * Si une case contient déjà un chiffre (non nul), son domaine est restreint à cette valeur unique.
3. Définir les contraintes du Sudoku (les valeurs doivent être uniques dans chaque ligne, colonne et région 3×3). Pour cela, utilisez la méthode addConstraint avec AllDifferentConstraint() pour assurer ces contraintes.
4. Résoudre le problème. Pour cela utilisez la méthode getSolutions() pour obtenir toutes les solutions possibles.
5. Gérer les solutions
  * Si des solutions existent, les stocker dans une liste.
  * Sinon, indiquer qu’aucune solution n’a été trouvée.

In [ ]:
from constraint import *

def solve_sudoku_csp(grid):
    problem = Problem()
    
    # 定义变量（网格中的每个单元格）及其取值范围（1-9）
    # 如果格子是 0（空），范围是 1-9；如果已有数字，范围就是该数字本身
    for row in range(9):
        for col in range(9):
            if grid[row][col] == 0:
                # addVariable(变量名, 值域)
                problem.addVariable((row, col), range(1, 10)) 
            else:
                problem.addVariable((row, col), [grid[row][col]])

    # 定义约束条件
    # 对每一行、每一列应用 "AllDifferent"（全互异）约束
    for i in range(9):
        # 行约束：第 i 行的所有列 (i, 0) 到 (i, 8) 的值必须不同
        problem.addConstraint(AllDifferentConstraint(), [(i, j) for j in range(9)])
        # i=0 [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8)]
        
        # 列约束：第 i 列的所有行 (0, i) 到 (8, i) 的值必须不同
        problem.addConstraint(AllDifferentConstraint(), [(j, i) for j in range(9)])
        # i=0 [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0)]

    # 对每个 3x3 九宫格应用约束
    for i in range(0, 9, 3):       # 遍历九宫格的起始行：0, 3, 6
        for j in range(0, 9, 3):   # 遍历九宫格的起始列：0, 3, 6
            # 收集该九宫格内的9个坐标并应用约束
            problem.addConstraint(AllDifferentConstraint(),
                                  [(i + a, j + b) for a in range(3) for b in range(3)])

    # 获取解
    solutions = problem.getSolutions()
    
    if solutions:  # 检查是否找到了解
        # 将第一个解（字典格式）转换为二维列表
        first_solution = solutions[0]
        # 初始化一个 9x9 的全 0 网格
        result_grid = [[0 for _ in range(9)] for _ in range(9)]
        
        # 遍历字典填入网格
        for (row, col), value in first_solution.items():
            result_grid[row][col] = value
        return result_grid
    else:
        return None  # 如果未找到解则返回 None


### **Exercice 5**
Utilisez votre fonction pour resoudre le jeu suivant.

In [8]:
# CSP 调用示例：0 表示空格
grid = [[5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]]


In [ ]:
# 调用方式：solve_sudoku_csp 返回解列表，通常取第 1 个解展示

print('Initial Sudoku')
visualize_sudoku(grid)





solution = solve_sudoku_csp(grid)
print('Solution Sudoku')
visualize_sudoku(solution)


5,3,4,6,7,8,9,1,2
6,7,2,1,9,5,3,4,8
1,9,8,3,4,2,5,6,7
8,5,9,7,6,1,4,2,3
4,2,6,8,5,3,7,9,1
7,1,3,9,2,4,8,5,6
9,6,1,5,3,7,2,8,4
2,8,7,4,1,9,6,3,5
3,4,5,2,8,6,1,7,9


# **Partie 2 : Comparaison CSP vs SAT**

Nous allons comparer les deux solveurs sur 10 sudokus differents. Pour cela, voici 10 grilles.

In [10]:
grid1 = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

In [11]:
grid2 = [
    [0, 0, 3, 0, 2, 0, 6, 0, 0],
    [9, 0, 0, 3, 0, 5, 0, 0, 1],
    [0, 0, 1, 8, 0, 6, 4, 0, 0],
    [0, 0, 8, 1, 0, 2, 9, 0, 0],
    [7, 0, 0, 0, 0, 0, 0, 0, 8],
    [0, 0, 6, 7, 0, 8, 2, 0, 0],
    [0, 0, 2, 6, 0, 9, 5, 0, 0],
    [8, 0, 0, 2, 0, 3, 0, 0, 9],
    [0, 0, 5, 0, 1, 0, 3, 0, 0]
]

In [12]:
grid3 = [
    [0, 2, 0, 6, 0, 8, 0, 0, 0],
    [5, 8, 0, 0, 0, 9, 7, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 0],
    [3, 7, 0, 0, 0, 0, 5, 0, 0],
    [6, 0, 0, 0, 0, 0, 0, 0, 4],
    [0, 0, 8, 0, 0, 0, 0, 1, 3],
    [0, 0, 0, 0, 2, 0, 0, 0, 0],
    [0, 0, 9, 8, 0, 0, 0, 3, 6],
    [0, 0, 0, 3, 0, 6, 0, 9, 0]
]

In [13]:
grid4 = [
    [0, 0, 5, 3, 0, 0, 0, 0, 0],
    [8, 0, 0, 0, 0, 0, 0, 2, 0],
    [0, 7, 0, 0, 1, 0, 5, 0, 0],
    [4, 0, 0, 0, 0, 5, 3, 0, 0],
    [0, 1, 0, 0, 7, 0, 0, 0, 6],
    [0, 0, 3, 2, 0, 0, 0, 8, 0],
    [0, 6, 0, 5, 0, 0, 0, 0, 9],
    [0, 0, 4, 0, 0, 0, 0, 3, 0],
    [0, 0, 0, 0, 0, 9, 7, 0, 0]
]

In [14]:
grid5 = [
    [8, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 3, 6, 0, 0, 0, 0, 0],
    [0, 7, 0, 0, 9, 0, 2, 0, 0],
    [0, 5, 0, 0, 0, 7, 0, 0, 0],
    [0, 0, 0, 0, 4, 5, 7, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 3, 0],
    [0, 0, 1, 0, 0, 0, 0, 6, 8],
    [0, 0, 8, 5, 0, 0, 0, 1, 0],
    [0, 9, 0, 0, 0, 0, 4, 0, 0]
]

In [15]:
grid6 = [
    [0, 0, 0, 6, 0, 0, 4, 0, 0],
    [7, 0, 0, 0, 0, 3, 6, 0, 0],
    [0, 0, 0, 0, 9, 1, 0, 8, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 5, 0, 1, 8, 0, 0, 0, 3],
    [0, 0, 0, 3, 0, 6, 0, 4, 5],
    [0, 4, 0, 2, 0, 0, 0, 6, 0],
    [9, 0, 3, 0, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 1, 0, 0]
]

In [16]:
grid7 = [
    [0, 2, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 6, 0, 0, 0, 0, 3],
    [0, 7, 4, 0, 8, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 2],
    [0, 8, 0, 0, 4, 0, 0, 1, 0],
    [6, 0, 0, 5, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 7, 8, 0],
    [5, 0, 0, 0, 0, 9, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 4, 0]
]

In [17]:
grid8 = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 8, 5],
    [0, 0, 1, 0, 2, 0, 0, 0, 0],
    [0, 0, 0, 5, 0, 7, 0, 0, 0],
    [0, 0, 4, 0, 0, 0, 1, 0, 0],
    [0, 9, 0, 0, 0, 0, 0, 0, 0],
    [5, 0, 0, 0, 0, 0, 0, 7, 3],
    [0, 0, 2, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 9]
]

In [18]:
grid9 = [
    [0, 0, 0, 2, 6, 0, 7, 0, 1],
    [6, 8, 0, 0, 7, 0, 0, 9, 0],
    [1, 9, 0, 0, 0, 4, 5, 0, 0],
    [8, 2, 0, 1, 0, 0, 0, 4, 0],
    [0, 0, 4, 6, 0, 2, 9, 0, 0],
    [0, 5, 0, 0, 0, 3, 0, 2, 8],
    [0, 0, 9, 3, 0, 0, 0, 7, 4],
    [0, 4, 0, 0, 5, 0, 0, 3, 6],
    [7, 0, 3, 0, 1, 8, 0, 0, 0]
]

In [19]:
grid10 = [
    [0, 2, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 6, 0, 0, 0, 0, 3],
    [0, 7, 4, 0, 8, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 2],
    [0, 8, 0, 0, 4, 0, 0, 1, 0],
    [6, 0, 0, 5, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 7, 8, 0],
    [5, 0, 0, 0, 0, 9, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 4, 0]
]

### **Exercice 6**

Créez une fonction comparer_solveurs qui prend comme entrée une liste de grilles, résout tous les jeux avec les deux solveurs, enregistre le temps d'exécution à chaque fois, et retourne les résultats sous forme de DataFrame.

Utilisez cette fonction pour résoudre tous les jeux et affichez ensuite les résultats.

In [20]:
import time
import pandas as pd
import copy
from tabulate import tabulate

def comparer_solveurs(grids):
    """
    比较 SAT 和 CSP 求解器在给定数独网格列表上的性能。
    Compare les performances des solveurs SAT et CSP sur une liste de grilles.
    """
    results = []
    
    for i, original_grid in enumerate(grids):
        # 使用深拷贝以确保每个求解器都使用干净的初始网格
        # Utiliser deepcopy pour s'assurer que chaque solveur utilise une grille propre
        grid_sat = copy.deepcopy(original_grid)
        grid_csp = copy.deepcopy(original_grid)
        
        # --- SAT Solver ---
        start_sat = time.time()
        solve_sudoku_sat(grid_sat) # 确保 solve_sudoku_sat 已在前面定义
        end_sat = time.time()
        time_sat = end_sat - start_sat
        
        # --- CSP Solver ---
        start_csp = time.time()
        solve_sudoku_csp(grid_csp) # 确保 solve_sudoku_csp 已在前面定义
        end_csp = time.time()
        time_csp = end_csp - start_csp
        
        # 记录结果 / Enregistrer les résultats
        results.append({
            "Sudoku": f"Grid {i+1}",
            "SAT Time (s)": time_sat,
            "CSP Time (s)": time_csp,
            "Diff (CSP-SAT)": time_csp - time_sat
        })
    
    return pd.DataFrame(results)

# 1. 准备网格列表 (确保 grid1 到 grid10 已经在之前的单元格中运行并定义)
# Préparer la liste des grilles
all_grids = [grid1, grid2, grid3, grid4, grid5, grid6, grid7, grid8, grid9, grid10]

# 2. 运行比较函数
# Exécuter la fonction de comparaison
df_results = comparer_solveurs(all_grids)


In [21]:
from tabulate import tabulate
print(tabulate(df_results, headers = 'keys', tablefmt = 'pretty'))

+---+---------+----------------------+-----------------------+-----------------------+
|   | Sudoku  |     SAT Time (s)     |     CSP Time (s)      |    Diff (CSP-SAT)     |
+---+---------+----------------------+-----------------------+-----------------------+
| 0 | Grid 1  | 0.011604547500610352 | 0.0026841163635253906 | -0.008920431137084961 |
| 1 | Grid 2  | 0.006568431854248047 | 0.002511262893676758  | -0.004057168960571289 |
| 2 | Grid 3  | 0.04494357109069824  |  0.08258199691772461  |  0.03763842582702637  |
| 3 | Grid 4  | 0.20842313766479492  |  0.04167652130126953  |  -0.1667466163635254  |
| 4 | Grid 5  | 0.27785420417785645  |  0.5117542743682861   |  0.2339000701904297   |
| 5 | Grid 6  | 0.20944690704345703  |  0.2367267608642578   |  0.02727985382080078  |
| 6 | Grid 7  |  0.2091386318206787  |  0.4051826000213623   |  0.1960439682006836   |
| 7 | Grid 8  | 0.24759626388549805  |   1.917569875717163   |   1.669973611831665   |
| 8 | Grid 9  | 0.004856586456298828 | 0.00